# 4 Wrangling - Combining SF Data<a id='4_Wrangling_-_Combining_SF_Data'></a>

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Wrangling - Combining SF Data](#4_Wrangling_-_Combining_SF_Data)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load The Data](#4.4_Load_The_Data)
  * [4.5 Prepare data for merging](#4.5_Prepare_data_for_merging)
      * [4.5.1 Police Incident data](#4.5.1_Police_incident_data)
      * [4.5.2 311 Case data](#4.5.2_311_Case_data)
      * [4.5.3 Housing sales data](#4.5.3_Housing_sales_data)
  * [4.6 Merge The Data](#4.6_Merge_The_Data)
  * [4.7 Save data](#4.7_Save_data)

## 4.2 Introduction<a id='4.2_Introduction'></a>

In this notebook, we will combine all the data and prepare it for feature analysis and modeling.

At the end of this notebook, we will generate the following file:

  * SF_Combined_SFPD_311_Housing.csv : all SF police incident reports, 311 cases, and housing sales data aggregated by month and by neighborhood, from January 2018 up to and including September 2020, wherein each row is an observation with a distinct pairing on month-year and each column represents a possible feature to be used in modelling

## 4.3 Imports<a id='4.3_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## 4.4 Load The Data<a id='4.4_Load_The_Data'></a>

In [2]:
df_sfpd = pd.read_csv('data/SF_Police_Incidents_aggregated.csv')

In [3]:
df_311 = pd.read_csv('data/311_Cases_aggregated.csv')

In [4]:
df_sales = pd.read_csv('data/Redfin_SF_sales_adjusted_neighborhood.csv')

In [5]:
print('sfpd incidents shape: ')
print(df_sfpd.shape)
print('311 cases shape: ')
print(df_311.shape)
print('housing sales shape: ')
print(df_sales.shape)

sfpd incidents shape: 
(157056, 7)
311 cases shape: 
(374480, 9)
housing sales shape: 
(3359, 9)


In [6]:
df_sfpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157056 entries, 0 to 157055
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Incident Year Month      157056 non-null  int64 
 1   Analysis Neighborhood    157056 non-null  object
 2   Report Type Description  157056 non-null  object
 3   Incident Category        157056 non-null  object
 4   Incident Description     157056 non-null  object
 5   Police District          157056 non-null  object
 6   Incident Count           157056 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.4+ MB


In [7]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374480 entries, 0 to 374479
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Opened Year Month  374480 non-null  int64 
 1   Source             374480 non-null  object
 2   Neighborhood       374480 non-null  object
 3   Police District    374480 non-null  object
 4   Status             374480 non-null  object
 5   Category           374480 non-null  object
 6   Request Type       374480 non-null  object
 7   Has Media          374480 non-null  bool  
 8   Case Count         374480 non-null  int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 23.2+ MB


In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year Month          3359 non-null   int64  
 1   311 Neighborhood          3359 non-null   object 
 2   Analysis Neighborhood     3359 non-null   object 
 3   Median Sale Price         3359 non-null   float64
 4   Homes Sold                3359 non-null   int64  
 5   New Listings              3359 non-null   float64
 6   Inventory                 3359 non-null   float64
 7   Days on Market            3359 non-null   float64
 8   Average Sale To List Pct  3359 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 236.3+ KB


**Note: `df_sfpd`'s `Analysis Neighborhood` will match to `df_sales`'s `Analysis Neighborhood`. `df_311`'s `Neighborhood` will match to `df_sales`'s `311 Neighborhood`.**

In order to properly map `df_sfpd` with `df_311`, we need to match up their Neighborhoods.

In [9]:
neighborhood_mapper = pd.read_csv('data/Neighborhoods_Map.csv')

In [10]:
neighborhood_mapper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RedfinNeighborhood     130 non-null    object
 1   311 Neighborhood       130 non-null    object
 2   Analysis Neighborhood  130 non-null    object
dtypes: object(3)
memory usage: 3.2+ KB


## 4.5 Prepare data for merging<a id='4.5_Prepare_data_for_merging'></a>

Before we merge the data, let's think about what we want the end result to look like. Our objective here is to create a model using the dataset, and to do that, we will need to performing regression analysis. Thus, every record (row) should be an observation. In this case, since we are planning on performing an analysis over time for every neighborhood, every record will be its own distinct month-year, neighborhood pairing. Every observation will share the same set of features (columns). Thus, the dataset will be in a wide format.

Before we merge the data, we need to arrange each data frame accordingly.

### 4.5.1 Police Incident data<a id='4.5.1_Police_incident_data'></a>

In [11]:
df_sfpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157056 entries, 0 to 157055
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Incident Year Month      157056 non-null  int64 
 1   Analysis Neighborhood    157056 non-null  object
 2   Report Type Description  157056 non-null  object
 3   Incident Category        157056 non-null  object
 4   Incident Description     157056 non-null  object
 5   Police District          157056 non-null  object
 6   Incident Count           157056 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.4+ MB


In [12]:
# let's rename a few columns for consistency
df_sfpd.rename(columns={'Incident Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [13]:
df_sfpd.head()

,Year Month,Neighborhood,Report Type Description,Incident Category,Incident Description,Police District,Incident Count
0,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, From Locked Vehicle, >$950",Bayview,24
1,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, From Unlocked Vehicle, >$950",Bayview,3
2,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, $50-$200",Bayview,4
3,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, $50-$200",Mission,1
4,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, >$950",Bayview,3


In [14]:
# use this as a check
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].groupby(['Report Type Description','Incident Category'])['Incident Count'].sum()

Report Type Description  Incident Category  
Coplogic Initial         Larceny Theft          35
                         Lost Property           3
                         Malicious Mischief     11
Coplogic Supplement      Burglary                2
                         Larceny Theft          10
                                                ..
Vehicle Initial          Robbery                 1
                         Stolen Property         1
Vehicle Supplement       Motor Vehicle Theft     3
                         Other Offenses          1
                         Recovered Vehicle      33
Name: Incident Count, Length: 67, dtype: int64

In [15]:
# testing the pivot
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Report Type Description','Incident Category'],
                                values='Incident Count',
                                aggfunc='sum',
                                fill_value=0)

Report Type Description          Coplogic Initial                \
Incident Category                   Larceny Theft Lost Property   
Year Month Neighborhood                                           
201801     Bayview Hunters Point               35             3   

Report Type Description                             Coplogic Supplement  \
Incident Category                Malicious Mischief            Burglary   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point                 11                   2   

Report Type Description                        Initial                   \
Incident Category                Larceny Theft   Arson Assault Burglary   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point            10       5      59       29   

Report Type Description                                              ...  \
Incident Category                Courtesy Report Disorderly Conduct  ...   
Year Month Neighborhood                                              ...   
201801     Bayview Hunters Point               1                 25  ...   

Report Type Description             Vehicle Initial                      \
Incident Category                Malicious Mischief Motor Vehicle Theft   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point                  1                  37   

Report Type Description                                              \
Incident Category                Other Miscellaneous Other Offenses   
Year Month Neighborhood                                               
201801     Bayview Hunters Point                   1              3   

Report Type Description                                                     \
Incident Category                Recovered Vehicle Robbery Stolen Property   
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                 3       1               1   

Report Type Description           Vehicle Supplement                 \
Incident Category                Motor Vehicle Theft Other Offenses   
Year Month Neighborhood                                               
201801     Bayview Hunters Point                   3              1   

Report Type Description                             
Incident Category                Recovered Vehicle  
Year Month Neighborhood                             
201801     Bayview Hunters Point                33  

[1 rows x 67 columns]

In [16]:
# create the table
df_sfpd_table = df_sfpd.pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Report Type Description','Incident Category'],
                                values='Incident Count',
                                aggfunc='sum',
                                fill_value=0)

In [17]:
df_sfpd_table.shape

(1353, 172)

In [18]:
df_sfpd_table.head()

Report Type Description          Coplogic Initial                     \
Incident Category                        Burglary Disorderly Conduct   
Year Month Neighborhood                                                
201801     Bayview Hunters Point                0                  0   
           Bernal Heights                       0                  0   
           Castro/Upper Market                  0                  0   
           Chinatown                            0                  0   
           Excelsior                            0                  0   

Report Type Description                                            \
Incident Category                Forgery And Counterfeiting Fraud   
Year Month Neighborhood                                             
201801     Bayview Hunters Point                          0     0   
           Bernal Heights                                 0     0   
           Castro/Upper Market                            0     1   
           Chinatown                                      0     0   
           Excelsior                                      0     0   

Report Type Description                                       \
Incident Category                Larceny Theft Lost Property   
Year Month Neighborhood                                        
201801     Bayview Hunters Point            35             3   
           Bernal Heights                   23             3   
           Castro/Upper Market              64             8   
           Chinatown                        19             5   
           Excelsior                         5             5   

Report Type Description                             Coplogic Supplement  \
Incident Category                Malicious Mischief            Burglary   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point                 11                   2   
           Bernal Heights                         2                   0   
           Castro/Upper Market                    5                   2   
           Chinatown                              7                   0   
           Excelsior                              2                   1   

Report Type Description                                            ...  \
Incident Category                Forgery And Counterfeiting Fraud  ...   
Year Month Neighborhood                                            ...   
201801     Bayview Hunters Point                          0     0  ...   
           Bernal Heights                                 0     0  ...   
           Castro/Upper Market                            0     0  ...   
           Chinatown                                      0     0  ...   
           Excelsior                                      0     0  ...   

Report Type Description          Vehicle Supplement                 \
Incident Category                   Stolen Property Suspicious Occ   
Year Month Neighborhood                                              
201801     Bayview Hunters Point                  0              0   
           Bernal Heights                         0              0   
           Castro/Upper Market                    0              0   
           Chinatown                              0              0   
           Excelsior                              0              0   

Report Type Description                                                      \
Incident Category                Traffic Collision Traffic Violation Arrest   
Year Month Neighborhood                                                       
201801     Bayview Hunters Point                 0                        0   
           Bernal Heights                        0                        0   
           Castro/Upper Market                   0                        0   
           Chinatown                             0                        0   
           Excelsior                             

In [19]:
df_sfpd_table.describe()

Report Type Description Coplogic Initial                     \
Incident Category               Burglary Disorderly Conduct   
count                        1353.000000        1353.000000   
mean                            0.028086           0.000739   
std                             0.201571           0.027186   
min                             0.000000           0.000000   
25%                             0.000000           0.000000   
50%                             0.000000           0.000000   
75%                             0.000000           0.000000   
max                             3.000000           1.000000   

Report Type Description                                                        \
Incident Category       Forgery And Counterfeiting        Fraud Larceny Theft   
count                                  1353.000000  1353.000000   1353.000000   
mean                                      0.062823     0.240207     34.543237   
std                                       0.274212     0.821734     37.698741   
min                                       0.000000     0.000000      0.000000   
25%                                       0.000000     0.000000      9.000000   
50%                                       0.000000     0.000000     22.000000   
75%                                       0.000000     0.000000     47.000000   
max                                       3.000000    13.000000    237.000000   

Report Type Description                                  Coplogic Supplement  \
Incident Category       Lost Property Malicious Mischief            Burglary   
count                     1353.000000        1353.000000         1353.000000   
mean                         4.121212           4.361419            1.019217   
std                          6.513193           4.384469            1.710458   
min                          0.000000           0.000000            0.000000   
25%                          1.000000           1.000000            0.000000   
50%                          2.000000           3.000000            0.000000   
75%                          5.000000           6.000000            1.000000   
max                         49.000000          27.000000           20.000000   

Report Type Description                                          ...  \
Incident Category       Forgery And Counterfeiting        Fraud  ...   
count                                  1353.000000  1353.000000  ...   
mean                                      0.011086     0.023651  ...   
std                                       0.104746     0.174658  ...   
min                                       0.000000     0.000000  ...   
25%                                       0.000000     0.000000  ...   
50%                                       0.000000     0.000000  ...   
75%                                       0.000000     0.000000  ...   
max                                       1.000000     3.000000  ...   

Report Type Description Vehicle Supplement                                   \
Incident Category          Stolen Property Suspicious Occ Traffic Collision   
count                           1353.00000    1353.000000       1353.000000   
mean                               0.04065       0.000739          0.005913   
std                                0.21200       0.027186          0.085799   
min                                0.00000       0.000000          0.000000   
25%                                0.00000       0.000000          0.000000   
50%                                0.00000       0.000000          0.000000   
75%                                0.00000       0.000000          0.000000   
max                                2.00000       1.000000          2.000000   

Report Type Description                                        \
Incident Category       Traffic Violation Arrest    Vandalism   
count                                1353.000000  1353.000000   
mean                                    0.002217  

In [20]:
# note that our columns are currently hierarchical, let's flatten them
df_sfpd_table.columns

MultiIndex([(   'Coplogic Initial',                   'Burglary'),
            (   'Coplogic Initial',         'Disorderly Conduct'),
            (   'Coplogic Initial', 'Forgery And Counterfeiting'),
            (   'Coplogic Initial',                      'Fraud'),
            (   'Coplogic Initial',              'Larceny Theft'),
            (   'Coplogic Initial',              'Lost Property'),
            (   'Coplogic Initial',         'Malicious Mischief'),
            ('Coplogic Supplement',                   'Burglary'),
            ('Coplogic Supplement', 'Forgery And Counterfeiting'),
            ('Coplogic Supplement',                      'Fraud'),
            ...
            ( 'Vehicle Supplement',            'Stolen Property'),
            ( 'Vehicle Supplement',             'Suspicious Occ'),
            ( 'Vehicle Supplement',          'Traffic Collision'),
            ( 'Vehicle Supplement',   'Traffic Violation Arrest'),
            ( 'Vehicle Supplement',           

In [21]:
df_sfpd_table.columns = df_sfpd_table.columns.to_flat_index()

In [22]:
df_sfpd_table.head()

(Coplogic Initial, Burglary)  \
Year Month Neighborhood                                          
201801     Bayview Hunters Point                             0   
           Bernal Heights                                    0   
           Castro/Upper Market                               0   
           Chinatown                                         0   
           Excelsior                                         0   

                                  (Coplogic Initial, Disorderly Conduct)  \
Year Month Neighborhood                                                    
201801     Bayview Hunters Point                                       0   
           Bernal Heights                                              0   
           Castro/Upper Market                                         0   
           Chinatown                                                   0   
           Excelsior                                                   0   

                                  (Coplogic Initial, Forgery And Counterfeiting)  \
Year Month Neighborhood                                                            
201801     Bayview Hunters Point                                               0   
           Bernal Heights                                                      0   
           Castro/Upper Market                                                 0   
           Chinatown                                                           0   
           Excelsior                                                           0   

                                  (Coplogic Initial, Fraud)  \
Year Month Neighborhood                                       
201801     Bayview Hunters Point                          0   
           Bernal Heights                                 0   
           Castro/Upper Market                            1   
           Chinatown                                      0   
           Excelsior                                      0   

                                  (Coplogic Initial, Larceny Theft)  \
Year Month Neighborhood                                               
201801     Bayview Hunters Point                                 35   
           Bernal Heights                                        23   
           Castro/Upper Market                                   64   
           Chinatown                                             19   
           Excelsior                                              5   

                                  (Coplogic Initial, Lost Property)  \
Year Month Neighborhood                                               
201801     Bayview Hunters Point                                  3   
           Bernal Heights                                         3   
           Castro/Upper Market                                    8   
           Chinatown                                              5   
           Excelsior                                              5   

                                  (Coplogic Initial, Malicious Mischief)  \
Year Month Neighborhood                                                    
201801     Bayview Hunters Point                                      11   
           Bernal Heights                                              2   
           Castro/Upper Market                                         5   
           Chinatown                                                   7   
           Excelsior                                                   2   

                                  (Coplogic Supplement, Burglary)  \
Year Month Neighborhood                                             
201801     Bayview Hunters Point                                2   
           Bernal Heights                                       0   
           Castro/Upper Market                                  2   
           Chinatown                                            0   
           Excelsior                                            

### 4.5.2 311 Case data<a id='4.5.2_311_Case_data'></a>

In [23]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374480 entries, 0 to 374479
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Opened Year Month  374480 non-null  int64 
 1   Source             374480 non-null  object
 2   Neighborhood       374480 non-null  object
 3   Police District    374480 non-null  object
 4   Status             374480 non-null  object
 5   Category           374480 non-null  object
 6   Request Type       374480 non-null  object
 7   Has Media          374480 non-null  bool  
 8   Case Count         374480 non-null  int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 23.2+ MB


In [24]:
# join with neighborhood mapper to get the Analysis Neighborhood

# fetch distinct 311 neighborhoods
neighborhood_mapper_311 = neighborhood_mapper[['311 Neighborhood','Analysis Neighborhood']].drop_duplicates()

In [25]:
df_311_neighborhood = pd.merge( df_311, neighborhood_mapper_311, left_on='Neighborhood', right_on='311 Neighborhood', validate='many_to_one')

In [26]:
df_311_neighborhood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374480 entries, 0 to 374479
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Opened Year Month      374480 non-null  int64 
 1   Source                 374480 non-null  object
 2   Neighborhood           374480 non-null  object
 3   Police District        374480 non-null  object
 4   Status                 374480 non-null  object
 5   Category               374480 non-null  object
 6   Request Type           374480 non-null  object
 7   Has Media              374480 non-null  bool  
 8   Case Count             374480 non-null  int64 
 9   311 Neighborhood       374480 non-null  object
 10  Analysis Neighborhood  374480 non-null  object
dtypes: bool(1), int64(2), object(8)
memory usage: 31.8+ MB


In [27]:
# drop unneeded columns
df_311_neighborhood.drop(columns=['Neighborhood','311 Neighborhood'], inplace=True)

In [28]:
# let's rename a few columns for consistency
df_311_neighborhood.rename(columns={'Opened Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [29]:
df_311_neighborhood.head()

,Year Month,Source,Police District,Status,Category,Request Type,Has Media,Case Count,Neighborhood
0,201801,Integrated Agency,NORTHERN,Closed,Graffiti,Graffiti on Building,False,2,Hayes Valley
1,201801,Integrated Agency,NORTHERN,Closed,Street Defects,Pavement_Defect,False,1,Hayes Valley
2,201801,Integrated Agency,NORTHERN,Closed,Street and Sidewalk Cleaning,Bulky Items,False,2,Hayes Valley
3,201801,Mobile/Open311,NORTHERN,Closed,Blocked Street or SideWalk,Blocked_Sidewalk,False,1,Hayes Valley
4,201801,Mobile/Open311,NORTHERN,Closed,Damaged Property,Damaged Fire_Police_Callbox,True,1,Hayes Valley


In [30]:
# use this as a check
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')
                    & (df_311_neighborhood['Category']=='Graffiti')].groupby(['Source','Status','Has Media'])['Case Count'].sum()

Source             Status  Has Media
Integrated Agency  Closed  False          4
Mobile/Open311     Closed  False         11
                           True         183
Phone              Closed  False         12
Web                Closed  False         23
Name: Case Count, dtype: int64

In [31]:
# testing the pivot
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')
                    & (df_311_neighborhood['Category']=='Graffiti')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Source','Status','Category','Has Media'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0)

Source                  Integrated Agency Mobile/Open311          Phone  \
Status                             Closed         Closed         Closed   
Category                         Graffiti       Graffiti       Graffiti   
Has Media                           False          False True     False   
Year Month Neighborhood                                                   
201801     Hayes Valley                 4             11   183       12   

Source                       Web  
Status                    Closed  
Category                Graffiti  
Has Media                  False  
Year Month Neighborhood           
201801     Hayes Valley       23

In [32]:
# create the table
df_311_table = df_311_neighborhood.pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Source','Status','Category','Has Media'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0)

In [33]:
df_311_table.shape

(1353, 368)

In [34]:
df_311_table.head()

Source                                     Email                   \
Status                                    Closed                    
Category                         General Request Illegal Postings   
Has Media                                  False            False   
Year Month Neighborhood                                             
201801     Bayview Hunters Point               0                0   
           Bernal Heights                      0                0   
           Castro/Upper Market                 0                0   
           Chinatown                           0                0   
           Excelsior                           0                0   

Source                                                                      \
Status                                                                       
Category                         Parking Enforcement Rec and Park Requests   
Has Media                                      False                 False   
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                   0                     0   
           Bernal Heights                          0                     0   
           Castro/Upper Market                     0                     0   
           Chinatown                               0                     0   
           Excelsior                               0                     0   

Source                                                                      \
Status                                                                       
Category                         Sewer Issues Street and Sidewalk Cleaning   
Has Media                               False                        False   
Year Month Neighborhood                                                      
201801     Bayview Hunters Point            0                            0   
           Bernal Heights                   0                            0   
           Castro/Upper Market              0                            0   
           Chinatown                        0                            0   
           Excelsior                        0                            0   

Source                                            Integrated Agency  \
Status                                                       Closed   
Category                         Tree Maintenance  Damaged Property   
Has Media                                   False             False   
Year Month Neighborhood                                               
201801     Bayview Hunters Point                0                 0   
           Bernal Heights                       0                 0   
           Castro/Upper Market                  0                 0   
           Chinatown                            0                 0   
           Excelsior                            0                 0   

Source                                                        ...         Web  \
Status                                                        ...        Open   
Category                         Encampments General Request  ... Sign Repair   
Has Media                              False           False  ...       False   
Year Month Neighborhood                                       ...               
201801     Bayview Hunters Point           4              11  ...           0   
           Bernal Heights                  0               2  ...           0   
           Castro/Upper Market             0               3  ...           0   
           Chinatown                       0               5  ...           0   
           Excelsior                       0               1  ...           0   

Source                                                       \
Status                                                        
Category                               Street Defects         
Has Media                        Tru

In [35]:
df_311_table.describe()

Source              Email                                       \
Status             Closed                                        
Category  General Request Illegal Postings Parking Enforcement   
Has Media           False            False               False   
count         1353.000000      1353.000000         1353.000000   
mean             0.000739         0.000739            0.000739   
std              0.027186         0.027186            0.027186   
min              0.000000         0.000000            0.000000   
25%              0.000000         0.000000            0.000000   
50%              0.000000         0.000000            0.000000   
75%              0.000000         0.000000            0.000000   
max              1.000000         1.000000            1.000000   

Source                                                                     \
Status                                                                      
Category  Rec and Park Requests Sewer Issues Street and Sidewalk Cleaning   
Has Media                 False        False                        False   
count               1353.000000  1353.000000                  1353.000000   
mean                   0.000739     0.000739                     0.001478   
std                    0.027186     0.027186                     0.038433   
min                    0.000000     0.000000                     0.000000   
25%                    0.000000     0.000000                     0.000000   
50%                    0.000000     0.000000                     0.000000   
75%                    0.000000     0.000000                     0.000000   
max                    1.000000     1.000000                     1.000000   

Source                     Integrated Agency                               \
Status                                Closed                                
Category  Tree Maintenance  Damaged Property  Encampments General Request   
Has Media            False             False        False           False   
count          1353.000000       1353.000000  1353.000000     1353.000000   
mean              0.000739          0.019217     0.085735        2.035477   
std               0.027186          0.142620     0.446928        2.842812   
min               0.000000          0.000000     0.000000        0.000000   
25%               0.000000          0.000000     0.000000        0.000000   
50%               0.000000          0.000000     0.000000        1.000000   
75%               0.000000          0.000000     0.000000        3.000000   
max               1.000000          2.000000     6.000000       25.000000   

Source     ...          Web                                           \
Status     ...         Open                                            
Category   ...  Sign Repair              Street Defects                
Has Media  ...        False        True           False        True    
count      ...  1353.000000  1353.000000    1353.000000  1353.000000   
mean       ...     0.033259     0.011826       0.044346     0.023651   
std        ...     0.213285     0.108141       0.219836     0.152016   
min        ...     0.000000     0.000000       0.000000     0.000000   
25%        ...     0.000000     0.000000       0.000000     0.000000   
50%        ...     0.000000     0.000000       0.000000     0.000000   
75%        ...     0.000000     0.000000       0.000000     0.000000   
max        ...     4.000000     1.000000       3.000000     1.000000   

Source                                                                         \
Status                                                                          
Category  Street and Sidewalk Cleaning              Streetlights                
Has Media                        False        True         False        True    
count                      1353.000000  1353.000000  1353.000000  1353.000000   
mean                          0.110126     0.063562     0.085735     0.028825   
std   

In [36]:
# note that our columns are currently hierarchical, let's flatten them
df_311_table.columns

MultiIndex([(            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            (            'Email', 'Closed', ...),
            ('Integrated Agency', 'Closed', ...),
            ('Integrated Agency', 'Closed', ...),
            ('Integrated Agency', 'Closed', ...),
            ...
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web',   'Open', ...),
            (              'Web', 

In [37]:
df_311_table.columns = df_311_table.columns.to_flat_index()

In [38]:
df_311_table.head()

(Email, Closed, General Request, False)  \
Year Month Neighborhood                                                     
201801     Bayview Hunters Point                                        0   
           Bernal Heights                                               0   
           Castro/Upper Market                                          0   
           Chinatown                                                    0   
           Excelsior                                                    0   

                                  (Email, Closed, Illegal Postings, False)  \
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                                         0   
           Bernal Heights                                                0   
           Castro/Upper Market                                           0   
           Chinatown                                                     0   
           Excelsior                                                     0   

                                  (Email, Closed, Parking Enforcement, False)  \
Year Month Neighborhood                                                         
201801     Bayview Hunters Point                                            0   
           Bernal Heights                                                   0   
           Castro/Upper Market                                              0   
           Chinatown                                                        0   
           Excelsior                                                        0   

                                  (Email, Closed, Rec and Park Requests, False)  \
Year Month Neighborhood                                                           
201801     Bayview Hunters Point                                              0   
           Bernal Heights                                                     0   
           Castro/Upper Market                                                0   
           Chinatown                                                          0   
           Excelsior                                                          0   

                                  (Email, Closed, Sewer Issues, False)  \
Year Month Neighborhood                                                  
201801     Bayview Hunters Point                                     0   
           Bernal Heights                                            0   
           Castro/Upper Market                                       0   
           Chinatown                                                 0   
           Excelsior                                                 0   

                                  (Email, Closed, Street and Sidewalk Cleaning, False)  \
Year Month Neighborhood                                                                  
201801     Bayview Hunters Point                                                  0      
           Bernal Heights                                                         0      
           Castro/Upper Market                                                    0      
           Chinatown                                                              0      
           Excelsior                                                              0      

                                  (Email, Closed, Tree Maintenance, False)  \
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                                         0   
           Bernal Heights                                                0   
           Castro/Upper Market                                           0   
           Chinatown                                                     0   
           Excelsior                                                     0   

                                  (Integrated Agency, Closed, Damaged Property, False)  \
Year Month

### 4.5.3 Housing sales data<a id='4.5.3_Housing_sales_data'></a>

In [39]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year Month          3359 non-null   int64  
 1   311 Neighborhood          3359 non-null   object 
 2   Analysis Neighborhood     3359 non-null   object 
 3   Median Sale Price         3359 non-null   float64
 4   Homes Sold                3359 non-null   int64  
 5   New Listings              3359 non-null   float64
 6   Inventory                 3359 non-null   float64
 7   Days on Market            3359 non-null   float64
 8   Average Sale To List Pct  3359 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 236.3+ KB


In [40]:
sales_cols_to_groupby = ['Sales Year Month', 'Analysis Neighborhood']
sales_cols_to_agg = ['Median Sale Price', 'Homes Sold', 'New Listings', 'Inventory', 'Days on Market', 'Average Sale To List Pct']
sales_cols_agg_methods = ['mean', 'sum', 'sum', 'sum', 'mean', 'mean']
sales_agg_methods_to_use = dict(zip(sales_cols_to_agg, sales_cols_agg_methods))
aggregated_sales = df_sales.groupby(sales_cols_to_groupby).agg(sales_agg_methods_to_use)
aggregated_sales = aggregated_sales.reset_index()

In [41]:
aggregated_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192 entries, 0 to 1191
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year Month          1192 non-null   int64  
 1   Analysis Neighborhood     1192 non-null   object 
 2   Median Sale Price         1192 non-null   float64
 3   Homes Sold                1192 non-null   int64  
 4   New Listings              1192 non-null   float64
 5   Inventory                 1192 non-null   float64
 6   Days on Market            1192 non-null   float64
 7   Average Sale To List Pct  1192 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 74.6+ KB


In [42]:
# let's rename a few columns for consistency
aggregated_sales.rename(columns={'Sales Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [43]:
aggregated_sales.head()

,Year Month,Neighborhood,Median Sale Price,Homes Sold,New Listings,Inventory,Days on Market,Average Sale To List Pct
0,201801,Bayview Hunters Point,830000.0,60,50.0,34.0,36.333333,108.716667
1,201801,Bernal Heights,1538500.0,77,61.0,25.0,31.500000,107.212500
2,201801,Castro/Upper Market,1351000.0,36,29.0,12.0,23.666667,111.933333
3,201801,Chinatown,1500000.0,3,2.0,1.0,141.000000,97.900000
4,201801,Excelsior,985000.0,36,35.0,17.0,14.666667,116.733333


In [44]:
aggregated_sales.shape

(1192, 8)

**We notice that we have fewer rows in the sales (1192) than in the others (1353)**

In [45]:
# first, confirm there are no differences between sfpd and 311 , make use of the MultiIndex
print(df_sfpd_table.index.difference(df_311_table.index))
print(df_311_table.index.difference(df_sfpd_table.index))

MultiIndex([], names=['Year Month', 'Neighborhood'])
MultiIndex([], names=['Year Month', 'Neighborhood'])


In [46]:
# find the difference between 311 and sales
index_difference = df_311_table.index.difference(list(zip(aggregated_sales['Year Month'],aggregated_sales['Neighborhood'])))

In [47]:
index_difference

MultiIndex([(201801, 'Golden Gate Park'),
            (201801,     'Lincoln Park'),
            (201801,     'McLaren Park'),
            (201801,         'Presidio'),
            (201801,  'Treasure Island'),
            (201802, 'Golden Gate Park'),
            (201802,     'Lincoln Park'),
            (201802,     'McLaren Park'),
            (201802,         'Presidio'),
            (201802,  'Treasure Island'),
            ...
            (202008, 'Golden Gate Park'),
            (202008,     'Lincoln Park'),
            (202008,     'McLaren Park'),
            (202008,         'Presidio'),
            (202008,  'Treasure Island'),
            (202009, 'Golden Gate Park'),
            (202009,     'Lincoln Park'),
            (202009,     'McLaren Park'),
            (202009,         'Presidio'),
            (202009,  'Treasure Island')],
           names=['Year Month', 'Neighborhood'], length=161)

In [48]:
index_difference.get_level_values(1).unique()

Index(['Golden Gate Park', 'Lincoln Park', 'McLaren Park', 'Presidio',
       'Treasure Island', 'Chinatown'],
      dtype='object', name='Neighborhood')

It makes sense that there are no housing sales for the parks (Golden Gate Park, Lincoln Park, and McLaren Park) and for Treasure Island, so we can consider dropping that data in the police incident reports and the 311 cases, but let's make sure that the other regions (Presidio and Chinatown) do contain some housing sales data.

In [49]:
aggregated_sales[aggregated_sales['Neighborhood'].isin(index_difference.get_level_values(1).unique())]['Neighborhood'].value_counts()

Chinatown    26
Presidio     11
Name: Neighborhood, dtype: int64

Looks fine, so when we merge the data, we can just fill null values with 0.

## 4.6 Merge The Data<a id='4.6_Merge_The_Data'></a>

It's time to merge the data and see how it looks all together.

In [53]:
# remember that we have multi-indexes for sfpd and 311 but not for housing sales
combined_sfpd_311 = pd.merge(df_sfpd_table, df_311_table, left_index=True, right_index=True).reset_index()

In [56]:
combined_sfpd_311_sales = pd.merge(combined_sfpd_311, aggregated_sales, on=['Year Month','Neighborhood'], how='left').fillna(0)

In [57]:
combined_sfpd_311_sales.shape

(1353, 548)

In [58]:
combined_sfpd_311_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353 entries, 0 to 1352
Columns: 548 entries, Year Month to Average Sale To List Pct
dtypes: float64(6), int64(541), object(1)
memory usage: 5.7+ MB


In [59]:
combined_sfpd_311_sales.describe()

,Year Month,"(Coplogic Initial, Burglary)","(Coplogic Initial, Disorderly Conduct)","(Coplogic Initial, Forgery And Counterfeiting)","(Coplogic Initial, Fraud)","(Coplogic Initial, Larceny Theft)","(Coplogic Initial, Lost Property)","(Coplogic Initial, Malicious Mischief)","(Coplogic Supplement, Burglary)","(Coplogic Supplement, Forgery And Counterfeiting)",...,"(Web, Open, Streetlights, False)","(Web, Open, Streetlights, True)","(Web, Open, Tree Maintenance, False)","(Web, Open, Tree Maintenance, True)",Median Sale Price,Homes Sold,New Listings,Inventory,Days on Market,Average Sale To List Pct
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,...,1353.000000,1353.000000,1353.000000,1353.000000,1.353000e+03,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,201897.000000,0.028086,0.000739,0.062823,0.240207,34.543237,4.121212,4.361419,1.019217,0.011086,...,0.085735,0.028825,0.108647,0.092387,1.345813e+06,40.271249,55.450850,31.253511,27.573597,94.733398
std,78.786131,0.201571,0.027186,0.274212,0.821734,37.698741,6.513193,4.384469,1.710458,0.104746,...,0.326419,0.180146,0.407972,0.406540,7.621753e+05,47.317381,73.899475,55.998125,22.167308,35.316884
min,201801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,201809.000000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.006000e+06,11.000000,15.000000,6.000000,16.000000,101.050000
50%,201905.000000,0.000000,0.000000,0.000000,0.000000,22.000000,2.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.375000e+06,28.000000,36.000000,16.000000,23.000000,105.925000
75%,202001.000000,0.000000,0.000000,0.000000,0.000000,47.000000,5.000000,6.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.725000e+06,51.000000,68.000000,34.000000,34.500000,110.650000
max,202009.000000,3.000000,1.000000,3.000000,13.000000,237.000000,49.000000,27.000000,20.000000,1.000000,...,3.000000,2.000000,4.000000,4.000000,7.925000e+06,311.000000,564.000000,606.000000,253.000000,129.333333


## 4.7 Save data<a id='4.7_Save_data'></a>

In [60]:
datapath = 'data'

# create datapath if it doesn't exist
if not os.path.exists(datapath):
    os.mkdir(datapath)

In [62]:
# write combined data
datapath_combineddata = os.path.join(datapath, 'SF_Combined_SFPD_311_Housing.csv')
if not os.path.exists(datapath_combineddata):
    combined_sfpd_311_sales.to_csv(datapath_combineddata, index=False)